In [ ]:
import geopandas as gpd
from tqdm import tqdm
import time
import pandas as pd
from geopandas import sjoin

In [ ]:
# 🕒 Timer global
global_start = time.time()

# 📥 Étape 1 : Chargement des données
start = time.time()
print("📥 Chargement des fichiers...")

parcelles = gpd.read_file("parcelles_filtrées.geojson")
monuments = gpd.read_file("monumentshistoriques.geojson")
zones_nat = gpd.read_file("zones_naturelles.geojson")
inond = gpd.read_file("risques_inondation.geojson")
natura2000 = gpd.read_file("natura2000.geojson")

# 🗺️ Fonction de reprojection vers EPSG:2154
def to_2154(gdf, name):
    print(f"🔎 {name} CRS d'origine : {gdf.crs}")
    if gdf.crs != "EPSG:2154":
        gdf = gdf.to_crs(epsg=2154)
        print(f"✅ {name} reprojetée en EPSG:2154")
    else:
        print(f"✅ {name} déjà en EPSG:2154")
    return gdf

# 🔁 Appliquer la fonction à chaque couche
parcelles = to_2154(parcelles, "Parcelles")
monuments = to_2154(monuments, "Monuments historiques")
zones_nat = to_2154(zones_nat, "Zones naturelles")
inond = to_2154(inond, "Zones inondation")
natura2000 = to_2154(natura2000, "Natura 2000")

print(f"✅ Données chargées en {time.time() - start:.2f} sec")

# 🧭 Vérification des projections
assert parcelles.crs == monuments.crs == zones_nat.crs == inond.crs == natura2000.crs == "EPSG:2154", "❌ Projections différentes !"

# 🔍 Étape 2 : Monuments Historiques avec buffer + sjoin
start = time.time()
print("🏛️ Analyse des monuments historiques (buffer 500m)...")

monuments_buffer = monuments.copy()
monuments_buffer["geometry"] = monuments_buffer.geometry.buffer(500)
monuments_buffer = monuments_buffer[monuments_buffer.geometry.is_valid]

parcelles["monument_historique"] = "Non"
result = sjoin(parcelles, monuments_buffer, how="inner", predicate="intersects")
parcelles.loc[parcelles.index.isin(result.index), "monument_historique"] = "Oui"

print(f"✅ Monuments traités en {time.time() - start:.2f} sec")

# 🌿 Étape 3 : Zones Naturelles
start = time.time()
print("🌿 Analyse des zones naturelles...")

zones_nat = zones_nat[zones_nat.geometry.is_valid]
parcelles["zone_naturelle"] = "Non"
result = sjoin(parcelles, zones_nat, how="inner", predicate="intersects")
parcelles.loc[parcelles.index.isin(result.index), "zone_naturelle"] = "Oui"

print(f"✅ Zones naturelles traitées en {time.time() - start:.2f} sec")

# 🌊 Étape 4 : Risques d’inondation
start = time.time()
print("🌊 Analyse des zones d’inondation...")

inond = inond[inond.geometry.is_valid]
parcelles["risque_inondation"] = "Non"
result = sjoin(parcelles, inond, how="inner", predicate="intersects")
parcelles.loc[parcelles.index.isin(result.index), "risque_inondation"] = "Oui"

print(f"✅ Inondations traitées en {time.time() - start:.2f} sec")

# 🌱 Étape 5 : Natura2000
start = time.time()
print("🌱 Analyse des zones Natura2000...")

natura2000 = natura2000[natura2000.geometry.is_valid]
parcelles["natura2000"] = "Non"
result = sjoin(parcelles, natura2000, how="inner", predicate="intersects")
parcelles.loc[parcelles.index.isin(result.index), "natura2000"] = "Oui"

print(f"✅ Natura2000 traitée en {time.time() - start:.2f} sec")

# 💾 Étape finale : Export
print("💾 Export vers GeoJSON et Excel...")

parcelles.to_file("parcelles_analyse_zonage.geojson", driver="GeoJSON")
parcelles[['Code site', 'commune', 'section', 'numero', 'prefixe',
           'monument_historique', 'zone_naturelle', 'risque_inondation','natura2000']
         ].to_excel("parcelles_analyse_zonage.xlsx", index=False)

print(f"✅ Export terminé en {time.time() - start:.2f} sec")
print(f"🎉 Script complet exécuté en {time.time() - global_start:.2f} sec")

